Helper files from: 
- Author: Sebastian Raschka
- GitHub Repository: https://github.com/rasbt/deeplearning-models


AlexNet model settings and training from:
- Github specific file: https://colab.research.google.com/drive/1IN0HD7-ljlPFtsbstfxLSKWvg2y2ndmO?usp=sharing#scrollTo=jvbILdHidK6b

In [1]:
import os
import time
import random

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms
from torchsummary import summary

import matplotlib.pyplot as plt
from PIL import Image
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())
def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
True


In [2]:
##########################
### SETTINGS
##########################

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.0001
BATCH_SIZE = 256
NUM_EPOCHS = 40

# Architecture
NUM_CLASSES = 10

# Other
dtype = torch.float
device = torch.device("mps")
set_all_seeds(RANDOM_SEED)

# Deterministic behavior not yet supported by AdaptiveAvgPool2d
#set_deterministic()

import sys

sys.path.insert(0, "..") # to include ../helper_evaluate.py etc.

from helper_evaluate import compute_accuracy
from helper_data import get_dataloaders_mnist
from helper_train import train_classifier_simple_v1

### Set random seed ###
set_all_seeds(RANDOM_SEED)

In [3]:
##########################
### Dataset
##########################

train_transforms = transforms.Compose([transforms.RandomCrop((28, 28)),
                                       transforms.ToTensor()])

test_transforms = transforms.Compose([transforms.CenterCrop((28, 28)),
                                      transforms.ToTensor()])


train_loader, valid_loader, test_loader = get_dataloaders_mnist (
    batch_size=BATCH_SIZE, 
    num_workers=2, 
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    validation_fraction=0.1)

# Checking the dataset
print('Training Set:\n')
for images, labels in train_loader:  
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    print(labels[:10])
    break
    
# Checking the dataset
print('\nValidation Set:')
for images, labels in valid_loader:  
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    print(labels[:10])
    break

# Checking the dataset
print('\nTesting Set:')
for images, labels in train_loader:  
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    print(labels[:10])
    break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9912422/9912422 [00:00<00:00, 13001242.48it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 2253855.06it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw




100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 9823109.38it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 23783431.67it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

Training Set:



Image batch dimensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])
tensor([8, 3, 7, 7, 4, 7, 5, 4, 3, 7])

Validation Set:
Image batch dimensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])
tensor([0, 3, 7, 6, 1, 4, 2, 2, 0, 9])

Testing Set:
Image batch dimensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])
tensor([9, 3, 0, 3, 4, 2, 5, 1, 6, 5])


In [25]:
##########################
### MODEL
##########################
cfg = {
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512],
}

class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 1
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, stride=1, padding=1),
                           #nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        return nn.Sequential(*layers)

def VGG16():
    return VGG('VGG16')


torch.manual_seed(RANDOM_SEED)

model = VGG16()
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 28, 28]             640
              ReLU-2           [-1, 64, 28, 28]               0
            Conv2d-3           [-1, 64, 28, 28]          36,928
              ReLU-4           [-1, 64, 28, 28]               0
         MaxPool2d-5           [-1, 64, 14, 14]               0
            Conv2d-6          [-1, 128, 14, 14]          73,856
              ReLU-7          [-1, 128, 14, 14]               0
            Conv2d-8          [-1, 128, 14, 14]         147,584
              ReLU-9          [-1, 128, 14, 14]               0
        MaxPool2d-10            [-1, 128, 7, 7]               0
           Conv2d-11            [-1, 256, 7, 7]         295,168
             ReLU-12            [-1, 256, 7, 7]               0
           Conv2d-13            [-1, 256, 7, 7]         590,080
             ReLU-14            [-1, 25

In [27]:

model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

In [28]:
log_dict = train_classifier_simple_v1(num_epochs=NUM_EPOCHS, model=model, 
                                      optimizer=optimizer, device=device, 
                                      train_loader=train_loader, valid_loader=valid_loader, 
                                      logging_interval=25)

Epoch: 001/040 | Batch 0000/0210 | Loss: 2.3040
Epoch: 001/040 | Batch 0025/0210 | Loss: 2.3025
Epoch: 001/040 | Batch 0050/0210 | Loss: 2.3009
Epoch: 001/040 | Batch 0075/0210 | Loss: 2.3019
Epoch: 001/040 | Batch 0100/0210 | Loss: 2.2946
Epoch: 001/040 | Batch 0125/0210 | Loss: 2.2483
Epoch: 001/040 | Batch 0150/0210 | Loss: 1.9255
Epoch: 001/040 | Batch 0175/0210 | Loss: 1.7616
Epoch: 001/040 | Batch 0200/0210 | Loss: 1.7383
***Epoch: 001/040 | Train. Acc.: 38.638% | Loss: 1.524
***Epoch: 001/040 | Valid. Acc.: 39.767% | Loss: 1.495
Time elapsed: 1.10 min
Epoch: 002/040 | Batch 0000/0210 | Loss: 1.5812
Epoch: 002/040 | Batch 0025/0210 | Loss: 1.6041
Epoch: 002/040 | Batch 0050/0210 | Loss: 1.2389
Epoch: 002/040 | Batch 0075/0210 | Loss: 1.4531
Epoch: 002/040 | Batch 0100/0210 | Loss: 1.2111
Epoch: 002/040 | Batch 0125/0210 | Loss: 1.2392
Epoch: 002/040 | Batch 0150/0210 | Loss: 1.1135
Epoch: 002/040 | Batch 0175/0210 | Loss: 1.0927
Epoch: 002/040 | Batch 0200/0210 | Loss: 1.1168
***

KeyboardInterrupt: 

In [ ]:
with torch.set_grad_enabled(False):
    
    train_acc = compute_accuracy(model=model,
                                 data_loader=test_loader,
                                 device=device)
    
    test_acc = compute_accuracy(model=model,
                                data_loader=test_loader,
                                device=device)
    
    valid_acc = compute_accuracy(model=model,
                                 data_loader=valid_loader,
                                 device=device)
    

print(f'Train ACC: {valid_acc:.2f}%')
print(f'Validation ACC: {valid_acc:.2f}%')
print(f'Test ACC: {test_acc:.2f}%')

# EXTRACT WEIGHTS


In [ ]:
params = [(name, p.data.cpu().numpy()) for (name, p) in model.named_parameters()]

In [ ]:
path = "../../files/preload/MNIST/VGG16/GG16/GG16/GG16/GG16/GG16/GG16/GG16"

In [ ]:
for i in params:
    name = i[0].split(".")
    for ii in range(3):
        file_name = name[2]+str(int(name[1])+1)+"_"+str(ii)
        np.savetxt(fname=f"{path}/{file_name}", delimiter=" ", X=i[1].flatten())

In [ ]:

# Write the used inputs and outputs
#np.savetxt(fname=f"{path}/input_0", delimiter=" ", X=images.cuda().view(-1, 3 * 33 * 33).tolist())
#np.savetxt(fname=f"{path}/outputlayer8_0", delimiter=" ", X=model.output(images.cuda().view(-1, 3, 33, 33)).data.cpu().view(-1))